# PP-Tracking之手把手玩转多目标跟踪

**PP-Tracking**是基于飞桨深度学习框架的业界首个开源实时跟踪系统。针对实际业务的难点痛点，PP-Tracking内置行人车辆跟踪、跨镜头跟踪、多类别跟踪、小目标跟踪及流量计数等能力与产业应用，同时提供可视化开发界面。模型集成多目标跟踪，目标检测，ReID轻量级算法，进一步提升PP-Tracking在服务器端部署性能。同时支持python，C++部署，适配Linux，Nvidia Jetson多平台环境。

![](https://ai-studio-static-online.cdn.bcebos.com/c8ffcde8c1974a69863806e3aeaf81c6680f3a8de50d4d56be9a14f155979f9e)

![](https://ai-studio-static-online.cdn.bcebos.com/ad76397a0c954b86b024a4330343e073e8ad0c1261da4a29be2a0f3bdbb196c6)
![](https://ai-studio-static-online.cdn.bcebos.com/38c90f74410041a3964e4f52b98f3ee881b443b6c7264f6ebd063b3c03ff8e4c)


在如下示例中，将介绍如何使用示例代码基于您在BML中已创建的数据集来完成单镜头跟踪模型的训练，评估和推理。以及多镜头的部署。



# 环境准备
下载PaddleDetection并且安装环境requirements

In [ ]:
!tar xvf PaddleDetection.tar

In [ ]:
# Download PaddleDetection
#!git clone https://gitee.com/paddlepaddle/PaddleDetection.git -b develop

In [ ]:
!pip install --upgrade pip -i https://mirror.baidu.com/pypi/simple
!pip install paddlepaddle-gpu==2.2.0rc0.post101 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html -i https://mirror.baidu.com/pypi/simple
!cd PaddleDetection/ && pip install pycocotools && pip install -r requirements.txt && python setup.py install

# 数据集准备

MOT16数据集：MOT16数据集是在2016年提出来的用于衡量多目标跟踪检测和跟踪方法标准的数据集，专门用于行人跟踪。官网地址是：https://motchallenge.net/

det.txt
这个文件中存储了图片的检测框的信息(检测得到的信息文件)，部分内容展示如下：

```
1,-1,1359.1,413.27,120.26,362.77,2.3092,-1,-1,-1
1,-1,571.03,402.13,104.56,315.68,1.5028,-1,-1,-1
```
从左到右分别代表：
* frame: 第几帧图片
* id: 这个检测框分配的id，在这里都是-1代表没有id信息
* bbox(四位): 分别是左上角坐标和长宽
* conf：这个bbox包含物体的置信度，可以看到并不是传统意义的0-1，分数越高代表置信度越高
* MOT3D(x,y,z): 是在MOT3D中使用到的内容，这里关心的是MOT2D，所以都设置为-1


#### 修改mot的配置文件，文件目录如下  
整理之前：
```
MOT16
  └——————train
  └——————test

整理之后：  
MOT16
   |——————images
   |        └——————train
   |        └——————test
   └——————labels_with_ids
            └——————train
```

详细参考[MOT数据准备文档](https://github.com/PaddlePaddle/PaddleDetection/blob/develop/docs/tutorials/PrepareMOTDataSet_cn.md)

In [ ]:
# 网速不好可以自行下载上传,解压
!cd ./data && wget https://bj.bcebos.com/v1/paddledet/data/mot/demo/MOT16.zip
!mv ./data/MOT16.zip ./PaddleDetection/dataset/mot
!cd ./PaddleDetection/dataset/mot && unzip MOT16.zip
# 生成labels_with_ids
!cd ./PaddleDetection/dataset/mot/MOT16 && mkdir -p images
!cd ./PaddleDetection/dataset/mot/MOT16 && mv ./train ./images && mv ./test ./images
!cd ./PaddleDetection/dataset/mot && python gen_labels_MOT.py
# 生成mot16.train文件并且复制到 image_lists下面
import glob
import os.path as osp
image_list = []
for seq in sorted(glob.glob('PaddleDetection/dataset/mot/MOT16/images/train/*')):
    for image in glob.glob(osp.join(seq, "img1")+'/*.jpg'):
        image = image.replace('PaddleDetection/dataset/mot/','')
        image_list.append(image)
with open('mot16.train','w') as image_list_file:
    image_list_file.write(str.join('\n',image_list))

!mkdir -p ./PaddleDetection/dataset/mot/image_lists && cp -r mot16.train ./PaddleDetection/dataset/mot/image_lists

# step 1
#### 修改配置文件里面的数据集
添加在PaddleDetection/configs/mot/fairmot/fairmot_dla34_30e_1088x608.yml文件最后

```
... ...
# for MOT training
# for MOT training
TrainDataset:
  !MOTDataSet
    dataset_dir: dataset/mot
    image_lists: ['mot16.train']
    data_fields: ['image', 'gt_bbox', 'gt_class', 'gt_ide']

# for MOT evaluation
# If you want to change the MOT evaluation dataset, please modify 'data_root'
EvalMOTDataset:
  !MOTImageFolder
    dataset_dir: dataset/mot
    data_root: MOT16/images/train
    keep_ori_im: False # set True if save visualization images or video, or used in DeepSORT

# for MOT video inference
TestMOTDataset:
  !MOTImageFolder
    dataset_dir: dataset/mot
    keep_ori_im: True # set True if save visualization images or video
 ```

# step 2
#### 开始训练

使用MOT16-02序列作为训练数据，训练30epoch，V100环境下大约需要30分钟

In [4]:
!cd PaddleDetection/ && python -m paddle.distributed.launch --log_dir=./fairmot_dla34_30e_1088x608/ --gpus 0 tools/train.py -c configs/mot/fairmot/fairmot_dla34_30e_1088x608.yml

-----------  Configuration Arguments -----------
elastic_server: None
force: False
gpus: 0
heter_worker_num: None
heter_workers: 
host: None
http_port: None
ips: 127.0.0.1
job_id: None
log_dir: ./fairmot_dla34_30e_1088x608/
np: None
nproc_per_node: None
run_mode: None
scale: 0
server_num: None
servers: 
training_script: tools/train.py
training_script_args: ['-c', 'configs/mot/fairmot/fairmot_dla34_30e_1088x608.yml']
worker_num: None
workers: 
------------------------------------------------
WARNING 2022-01-07 16:51:28,232 launch.py:381] Not found distinct arguments and compiled with cuda or xpu. Default use collective mode
launch train in GPU mode!
INFO 2022-01-07 16:51:28,236 launch_utils.py:525] Local start 1 processes. First process distributed environment info (Only For Debug): 
    +=======================================================================================+
    |                        Distributed Envs                      Value                    |
    +-------------

# step3
#### 开始评估
为了方便我们下载训练好的模型进行eval
https://paddledet.bj.bcebos.com/models/mot/fairmot_dla34_30e_1088x608.pdparams

In [5]:
# download model.pdparams and put it into `output` directory
!mkdir -p PaddleDetection/output && cd PaddleDetection/output/ && wget https://bj.bcebos.com/v1/paddledet/models/mot/fairmot_dla34_30e_1088x608.pdparams 
!cd PaddleDetection && CUDA_VISIBLE_DEVICES=0 python tools/eval_mot.py -c configs/mot/fairmot/fairmot_dla34_30e_1088x608.yml -o weights=output/fairmot_dla34_30e_1088x608.pdparams

--2022-01-07 17:00:19--  https://bj.bcebos.com/v1/paddledet/models/mot/fairmot_dla34_30e_1088x608.pdparams
Resolving bj.bcebos.com (bj.bcebos.com)... 182.61.200.195, 182.61.200.229, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to bj.bcebos.com (bj.bcebos.com)|182.61.200.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130895015 (125M) [application/octet-stream]
Saving to: ‘fairmot_dla34_30e_1088x608.pdparams’

fairmot_dla34_30e_1 100%[===================>] 124.83M  31.6MB/s    in 4.0s    

2022-01-07 17:00:23 (30.9 MB/s) - ‘fairmot_dla34_30e_1088x608.pdparams’ saved [130895015/130895015]

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:130: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.

# step 4
#### 开始推理
使用下载好的模型进行推理，为了方便我们只推理了`dataset/mot/MOT16/images/test/MOT16-01/img1`下面的数据

跟踪输出视频保存在`output/mot_outputs/img1_vis.mp4` 

txt文件结果保存在`output/mot_results/img1.txt`,输出格式表示为frame_id, id, bbox_left, bbox_top, bbox_width, bbox_height, score, x, y, z 

In [6]:
!cd PaddleDetection/ && CUDA_VISIBLE_DEVICES=0 python tools/infer_mot.py -c configs/mot/fairmot/fairmot_dla34_30e_1088x608.yml -o weights=output/fairmot_dla34_30e_1088x608.pdparams --image_dir=dataset/mot/MOT16/images/test/MOT16-01/img1  --save_videos

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:130: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
W0107 17:01:31.576774  2159 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0107 17:01:31.582242  2159 device_context.cc:465] device: 0, cuDNN Version: 7.6.
[01/07 17:01:35] ppdet.utils.checkpoint INFO: Finish resuming model weights: output/fairmot_dla34_30e_1088x608.pdparams
[01/07 17:01:35] ppdet.engine.tracker INFO: Starting tracking folder dataset/mot/MOT16/images/test/MOT16-01/img1, found 450 images
[01/07 17:01:35] ppdet.engine.tracker INFO: Processing frame 0 (100000.00 fps)



# step 5
#### 导出模型


In [7]:
!cd PaddleDetection && CUDA_VISIBLE_DEVICES=0 python tools/export_model.py -c configs/mot/fairmot/fairmot_dla34_30e_1088x608.yml -o weights=output/fairmot_dla34_30e_1088x608.pdparams


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:130: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
[01/07 17:02:48] ppdet.utils.checkpoint INFO: Finish loading model weights: output/fairmot_dla34_30e_1088x608.pdparams
[01/07 17:02:48] ppdet.engine INFO: Export inference config file to output_inference/fairmot_dla34_30e_1088x608/infer_cfg.yml
W0107 17:02:51.943039  2357 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0107 17:02:51.943104  2357 device_context.cc:465] device: 0, cuDNN Version: 7.6.
[01/07 17:02:55] ppdet.engine INFO: Export model and saved in output_inference/fairm

# step 6
#### 使用导出的模型进行推理

PP-Tracking中在部署阶段提供了多种跟踪相关功能，例如流量计数，出入口统计，绘制跟踪轨迹等，具体使用方法可以参考[文档](https://github.com/PaddlePaddle/PaddleDetection/tree/release/2.3/deploy/pptracking/python#%E5%8F%82%E6%95%B0%E8%AF%B4%E6%98%8E)

In [8]:
!cd PaddleDetection && wget https://bj.bcebos.com/v1/paddledet/data/mot/demo/person.mp4 && wget https://bj.bcebos.com/v1/paddledet/data/mot/demo/entrance_count_demo.mp4
# 输出视频保存在output/person.mp4中
!cd PaddleDetection && python deploy/pptracking/python/mot_jde_infer.py --model_dir=output_inference/fairmot_dla34_30e_1088x608 --video_file=person.mp4 --device=GPU
!cd PaddleDetection && python deploy/pptracking/python/mot_jde_infer.py --model_dir=output_inference/fairmot_dla34_30e_1088x608 --do_entrance_counting --draw_center_traj --video_file=entrance_count_demo.mp4 --device=GPU

--2022-01-07 17:02:57--  https://bj.bcebos.com/v1/paddledet/data/mot/demo/person.mp4
Resolving bj.bcebos.com (bj.bcebos.com)... 182.61.200.229, 182.61.200.195, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to bj.bcebos.com (bj.bcebos.com)|182.61.200.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8850805 (8.4M) [video/mp4]
Saving to: ‘person.mp4’

person.mp4          100%[===================>]   8.44M  10.2MB/s    in 0.8s    

2022-01-07 17:02:58 (10.2 MB/s) - ‘person.mp4’ saved [8850805/8850805]

--2022-01-07 17:02:58--  https://bj.bcebos.com/v1/paddledet/data/mot/demo/entrance_count_demo.mp4
Resolving bj.bcebos.com (bj.bcebos.com)... 182.61.200.229, 182.61.200.195, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to bj.bcebos.com (bj.bcebos.com)|182.61.200.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6977441 (6.7M) [video/mp4]
Saving to: ‘entrance_count_demo.mp4’

entrance_count_demo 100%[===================>]   6.65M  4.71M

# MTMCT 跨镜跟踪体验

跨镜头多目标跟踪是对同一场景下的不同摄像头拍摄的视频进行多目标跟踪，是监控视频领域一个非常重要的研究课题。

相较于单镜头跟踪，跨镜跟踪将不同镜头获取到的跟踪轨迹进行融合，得到跨镜跟踪的输出轨迹。PP-Tracking选用DeepSORT方案实现跨镜跟踪，为了达到实时性选用了PaddleDetection自研的PP-YOLOv2和PP-PicoDet作为检测器，选用PaddleClas自研的轻量级网络PP-LCNet作为ReID模型。更多内容可参考[文档](https://github.com/PaddlePaddle/PaddleDetection/tree/develop/configs/mot/mtmct)

本项目展示城市主干道场景下的车辆跨镜跟踪预测流程，数据来自[AIC21](https://www.aicitychallenge.org/)开源数据集

# Step 1
#### 下载预测部署模型

首先我们下载目标检测和ReID预测模型，[下载地址](https://github.com/PaddlePaddle/PaddleDetection/tree/develop/configs/mot/mtmct#deepsort%E5%9C%A8-aic21-mtmctcityflow-%E8%BD%A6%E8%BE%86%E8%B7%A8%E5%A2%83%E8%B7%9F%E8%B8%AA%E6%95%B0%E6%8D%AE%E9%9B%86test%E9%9B%86%E4%B8%8A%E7%9A%84%E7%BB%93%E6%9E%9C)，然后统一放在`~/PaddleDetection/output_inference`下

In [ ]:
!wget https://paddledet.bj.bcebos.com/models/mot/deepsort/ppyolov2_r50vd_dcn_365e_aic21mtmct_vehicle.tar
!wget https://paddledet.bj.bcebos.com/models/mot/deepsort/deepsort_pplcnet_vehicle.tar
!cd ~/PaddleDetection/ && mkdir -p output_inference
!mv ppyolov2_r50vd_dcn_365e_aic21mtmct_vehicle.tar ~/PaddleDetection/output_inference
!mv deepsort_pplcnet_vehicle.tar ~/PaddleDetection/output_inference
!cd ~/PaddleDetection/output_inference && tar -xvf ppyolov2_r50vd_dcn_365e_aic21mtmct_vehicle.tar && tar -xvf deepsort_pplcnet_vehicle.tar

# Step 2
#### 跨镜跟踪预测

在完成模型下载后，需要修改`PaddleDetection/deploy/pptracking/python`路径下的`mtmct_cfg.yml`，这份配置文件中包含了跨镜跟踪中轨迹融合的相关参数。首先需要确定`cameras_bias`中对应的名称与输入视频名称对应；其次，我们本次项目使用轨迹融合中的通用方法，将zone和camera相关的方法设置为False。修改后配置如下：

```
# config for MTMCT
MTMCT: True
cameras_bias:
  c003: 0
  c004: 0
# 1.zone releated parameters
use_zone: False #True
zone_path: dataset/mot/aic21mtmct_vehicle/S06/zone
# 2.tricks parameters, can be used for other mtmct dataset
use_ff: True
use_rerank: True
# 3.camera releated parameters
use_camera: False #True
use_st_filter: False
# 4.zone releated parameters
use_roi: False #True
roi_dir: dataset/mot/aic21mtmct_vehicle/S06
```


配置完成后即可运行如下命令，输入视频为c003.mp4和c004.mp4两个不同视角的摄像头拍摄结果，跨镜跟踪输出视频保存在

In [ ]:
!wget https://bj.bcebos.com/v1/paddledet/data/mot/demo/mtmct-demo.tar && mv mtmct-demo.tar ~/PaddleDetection && cd ~/PaddleDetection && tar xvf mtmct-demo.tar 
!cd ~/PaddleDetection && python deploy/pptracking/python/mot_sde_infer.py --model_dir=output_inference/ppyolov2_r50vd_dcn_365e_aic21mtmct_vehicle/ --reid_model_dir=output_inference/deepsort_pplcnet_vehicle/ --mtmct_dir=./mtmct-demo --device=GPU --mtmct_cfg=deploy/pptracking/python/mtmct_cfg.yml --scaled=True --save_mot_txts --save_images
